In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings("ignore");

In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import requests

In [8]:
url="https://www.amazon.in/s?k=laptop&i=computers&crid=1YDNORBH8ERL2&sprefix=laptop%2Ccomputers%2C372&ref=nb_sb_noss_2"

In [9]:
uClient = urlopen(url)
page_html = uClient.read()
uClient.close()

In [10]:
page_soup = BeautifulSoup(page_html, features='html')

In [11]:
print(page_soup)

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-in"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<!-- sp:end-feature:head-start -->
<!-- sp:feature:csm:head-open-part1 -->
<!-- sp:end-feature:csm:head-open-part1 -->
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="https://images-eu.ssl-images-amazon.com" rel="dns-prefetch"/>
<link href="https://m.media-amazon.com" rel="dns-prefetch"/>
<link href="https://completion.amazon.com" rel="dns-prefetch"/>
<!-- sp:end-feature:cs-optimization -->
<!-- sp:feature:csm:head-open-part2 -->
<!-- sp:end-feature:csm:head-open-part2 -->
<!-- sp:feature:aui-assets -->
<link href="https://m.media-amazon.com/images/I/21ok1pmLrnL._RC|51muT-wMT2L.css_.css?AUIClients/AmazonUI" rel="stylesheet"/>
<link href="https://m.media-amazon.com/images/I/11Q3J0BXRuL._RC|01ZTHTZObnL.css,41CH6lOLkAL.css,315p88chKeL.css,21+Rc

In [20]:
product_class_name = "puisg-col-inner"

In [21]:
containers = page_soup.find_all("div", {"class": product_class_name})

In [23]:
print(containers[0])

<div class="puisg-col-inner"><div class="a-section a-spacing-none aok-relative puis-status-badge-container s-list-status-badge-container"></div><div class="s-product-image-container aok-relative s-text-center s-image-overlay-grey puis-image-overlay-grey s-padding-left-small s-padding-right-small puis-flex-expand-height puis puis-vkh8i5perkddu2r7ogc2c0qfsb" data-cy="image-container"><div class="aok-relative"><span class="rush-component" data-component-type="s-product-image" data-render-id="rlc2rgbcq51972974okwxlc2gx" data-version-id="vkh8i5perkddu2r7ogc2c0qfsb"><a aria-hidden="true" class="a-link-normal s-no-outline" href="/HP-Pavilion-i5-1335U-Laptop-Touchscreen/dp/B0CJ2KWNNQ" tabindex="-1" target="_blank"><div class="a-section aok-relative s-image-fixed-height"><img alt='HP Pavilion x360, 13th Gen Intel Core i5-1335U (16GB DDR4, 512GB SSD) Touchscreen, 14"/35.6cm, FHD, Win 11, Office 21, Sil...' aria-hidden="true" class="s-image" data-image-index="1" data-image-latency="s-product-imag

In [24]:
import csv

# URL (example search for laptops on Amazon India)
url = "https://www.amazon.in/s?k=laptop&i=computers&crid=1YDNORBH8ERL2&sprefix=laptop%2Ccomputers%2C372&ref=nb_sb_noss_2"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/117.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9"
}

# Fetch page
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Find product containers
containers = soup.find_all("div", {"data-component-type": "s-search-result"})

# CSV setup
filename = "amazon_laptops.csv"
fields = ["title", "brand", "processor", "ram", "storage", "screen", 
          "resolution", "os", "office", "features", "link", "image"]

rows = []

for container in containers:
    try:
        # Title from alt text
        img_tag = container.find("img", class_="s-image")
        if not img_tag:
            continue
        title = img_tag["alt"]

        # Link
        a_tag = container.find("a", class_="a-link-normal")
        link = "https://www.amazon.in" + a_tag["href"] if a_tag else None

        # Image
        image_url = img_tag["src"]

        # Details dictionary
        details = {
            "brand": None,
            "processor": None,
            "ram": None,
            "storage": None,
            "screen": None,
            "resolution": None,
            "os": None,
            "office": None,
            "feature": []
        }

        # Split title by commas
        parts = title.split(",")
        for part in parts:
            part = part.strip()
            if any(b in part for b in ["HP", "Dell", "ASUS", "Lenovo", "Acer", "MSI"]):
                details["brand"] = part
            elif "Intel" in part or "Ryzen" in part:
                details["processor"] = part
            elif "GB" in part and "DDR" in part:
                details["ram"] = part
            elif "SSD" in part or "HDD" in part:
                details["storage"] = part
            elif '"' in part or "cm" in part:
                details["screen"] = part
            elif "FHD" in part or "UHD" in part or "OLED" in part:
                details["resolution"] = part
            elif "Win" in part:
                details["os"] = part
            elif "Office" in part:
                details["office"] = part
            else:
                details["feature"].append(part)

        rows.append([
            title,
            details["brand"],
            details["processor"],
            details["ram"],
            details["storage"],
            details["screen"],
            details["resolution"],
            details["os"],
            details["office"],
            ", ".join(details["feature"]),
            link,
            image_url
        ])

    except Exception as e:
        print("Skipping item due to error:", e)

# Write to CSV
with open(filename, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    writer.writerows(rows)

print(f"✅ Scraping complete. Data saved to {filename}")

✅ Scraping complete. Data saved to amazon_laptops.csv
